### How to create a good title for video?

In [ ]:
import pandas as pd

british_youtube = pd.read_csv("GBvideos.csv")
canadian_youtube = pd.read_csv("CAvideos.csv")
us_youtube = pd.read_csv("USvideos.csv")

videos = pd.concat([canadian_youtube, british_youtube,us_youtube])
videos.head()

### What makes videos popular?

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt


class CorrelationMatrix:
    def __init__(self, df):
        self.df = df

    def draw(self):
        corr_matrix = self.df[['views', 'likes', 'dislikes', 'comment_count']].corr()
        sn.heatmap(corr_matrix, annot=True)
        plt.show()


CorrelationMatrix(videos).draw()

In [ ]:
class YoutubeNormalizer:

    def __init__(self, csv):
        self.csv = csv

    def decide_about_popular(self):
        lots_views = self.csv.views.quantile(0.8)
        comments_median = self.csv.comment_count.quantile(0.6)
        lots_of_dislike = self.csv.dislikes.quantile(0.8)
        self.csv['popular'] = 0
        self.csv.loc[(comments_median < self.csv.comment_count) & (self.csv.views > lots_views), 'popular'] = 1
        self.csv.loc[(self.csv.dislikes > lots_of_dislike) & (self.csv.likes > self.csv.dislikes), 'popular'] = 1

    def get_normalized_data(self):
        self.csv.drop_duplicates(subset="title", keep='first', inplace=True)
        self.decide_about_popular()
        title_with_popular = self.csv[['title', 'popular']]
        return title_with_popular


normalized_data = YoutubeNormalizer(videos).get_normalized_data()
normalized_data.head()

### Title Normalization

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import Word
import warnings

warnings.filterwarnings('ignore')

nltk.download('wordnet')
nltk.download('stopwords')

stop = stopwords.words('english')
st = PorterStemmer()


class NormalizedTitle:
    def __init__(self, df):
        self.df = df

    @staticmethod
    def avg_word(sentence):
        words = sentence.split()
        return sum(len(word) for word in words) / len(words)

    def parameters_extraction(self):
        self.df['word_count'] = self.df['title'].apply(lambda x: len(str(x).split(" ")))
        self.df['char_count'] = self.df['title'].str.len()
        self.df['avg_word'] = self.df['title'].apply(lambda x: self.avg_word(x))
        self.df['stopwords'] = self.df['title'].apply(lambda x: len([x for x in x.split() if x in stop]))
        self.df['hastags'] = self.df['title'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
        self.df['numerics'] = self.df['title'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

    def text_preparation(self):
        self.df['title'] = self.df['title'].apply(lambda x: " ".join(x.lower() for x in x.split()))
        self.df['title'] = self.df['title'].str.replace('[^\w\s]', '')
        self.df['title'] = self.df['title'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
        freq = pd.Series(' '.join(self.df['title']).split()).value_counts()
        common_words = freq[freq > 250]
        common_words = list(common_words.index)
        self.df['title'] = self.df['title'].apply(lambda x: " ".join(x for x in x.split() if x not in common_words))
        rare_words = common_words = freq[freq < 5]
        rare_words = list(common_words.index)
        self.df['title'] = self.df['title'].apply(lambda x: " ".join(x for x in x.split() if x not in rare_words))
        self.df['title'] = self.df['title'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
        self.df['title'] = self.df['title'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

    def process(self):
        self.parameters_extraction()
        self.text_preparation()
        return self.df


normalized_title = NormalizedTitle(normalized_data).process()
normalized_title.head()

### Title Vectorization

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics, naive_bayes, svm, ensemble


class TitleVectorization:
    def __init__(self, df):
        self.df = df

    def process(self):
        train_df, test_df = train_test_split(
            self.df,
            test_size=0.1,
            stratify=self.df['popular'],
        )
        vectorizer = CountVectorizer(max_features=1000, lowercase=True, ngram_range=(1, 2), analyzer="word")
        vectorizer.fit(train_df)
        return train_df, test_df, vectorizer


(train_title_popularity, test_title_popularity, vectorizer) = TitleVectorization(normalized_title).process()

### Classification and metrics

In [ ]:
class Classification:
    def __init__(self, train_df, test_df, vectorizer, strategy):
        self.train_df = train_df
        self.test_df = test_df
        self.vectorizer = vectorizer
        if strategy == 'LogisticRegression':
            self.model = LogisticRegression(class_weight='balanced', dual=False)
        elif strategy == 'NAIVE BAYES':
            self.model = naive_bayes.MultinomialNB()
        elif strategy == 'SVM':
            self.model = svm.SVC()
        elif strategy == 'RANDOM FORREST':
            self.model = ensemble.RandomForestClassifier()

    def train(self):
        transformed = vectorizer.transform(self.train_df['title'])
        self.model.fit(transformed, self.train_df['popular'])

    def show_model_statistic(self):
        vectorized = vectorizer.transform(self.test_df['title'])
        predicted = self.model.predict(vectorized)
        target = self.test_df['popular']
        print(metrics.classification_report(target, predicted, digits=4))

### Logistic Regression

In [ ]:
logistic_regression_classification = Classification(train_df=train_title_popularity, test_df=test_title_popularity, vectorizer=vectorizer,
                                                    strategy='LogisticRegression')
logistic_regression_classification.train()
logistic_regression_classification.show_model_statistic()

### NAIVE BAYES

In [ ]:
naive_bayes_classification = Classification(train_df=train_title_popularity, test_df=test_title_popularity, vectorizer=vectorizer,
               strategy='NAIVE BAYES')
naive_bayes_classification.train()
naive_bayes_classification.show_model_statistic()

### SVM

In [ ]:
svm_classification = Classification(train_df=train_title_popularity, test_df=test_title_popularity, vectorizer=vectorizer,
               strategy='SVM')
svm_classification.train()
svm_classification.show_model_statistic()

### RANDOM FORREST

In [ ]:
random_forest_tree = Classification(train_df=train_title_popularity, test_df=test_title_popularity, vectorizer=vectorizer,
               strategy='RANDOM FORREST')
random_forest_tree.train()
random_forest_tree.show_model_statistic()